In [16]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier


  0%|          | 0/14676 [21:02<?, ?it/s]


In [ ]:
model_name = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv('/content/Preprocessed_data.csv')
df.head()

,tweet_with_emoji_meaning,label
0,پیرمرد وصیت احدی تاکید احدی درد دل نکنید بعدش ...,0
1,مجوز بده ملت ماشین استاندارد بتونن بیارن سوار...,0
2,دیت دختره دید زشتم می‌خواست پاشه بره آینهی دست...,1
3,اکیپ دخترونه هست پایه قراراست کلاس نمیذاره زود...,0
4,۵۰۰ نفری مراسم سالگرد پدر همسرم شرکت فوتی‌ی ۴۰...,0


In [ ]:
texts = df["tweet_with_emoji_meaning"].astype(str).tolist()
labels = df["label"].tolist()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def get_bert_embedding_pb(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[0, 0, :].cpu().numpy()
    return cls_embedding

In [ ]:
X = np.array([get_bert_embedding_pb(text) for text in tqdm(texts)])
y = np.array(labels)




  0%|          | 0/14676 [00:00<?, ?it/s]


  0%|          | 6/14676 [00:00<05:27, 44.78it/s]


  0%|          | 11/14676 [00:00<06:08, 39.74it/s]


  0%|          | 15/14676 [00:00<07:21, 33.20it/s]


  0%|          | 23/14676 [00:00<05:12, 46.91it/s]


  0%|          | 30/14676 [00:00<04:34, 53.45it/s]


  0%|          | 38/14676 [00:00<03:59, 61.06it/s]


  0%|          | 0/14676 [00:44<?, ?it/s]



  0%|          | 52/14676 [00:00<04:05, 59.52it/s]


  0%|          | 59/14676 [00:01<04:18, 56.56it/s]


  0%|          | 66/14676 [00:01<04:03, 60.08it/s]


  0%|          | 73/14676 [00:01<03:54, 62.22it/s]


  1%|          | 81/14676 [00:01<03:44, 65.15it/s]


  1%|          | 88/14676 [00:01<03:39, 66.40it/s]


  1%|          | 95/14676 [00:01<03:44, 64.86it/s]


  1%|          | 103/14676 [00:01<03:37, 66.92it/s]


  1%|          | 110/14676 [00:01<03:39, 66.50it/s]


  1%|          | 118/14676 [00:01<03:33, 68.34it/s]


  1%|          | 126/14676 [00:02<03:30, 69.25it/s]


  1%

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
models = {
    'Logistic Regression': LogisticRegression(),
    # 'MultinomialNB': MultinomialNB(),
    'SVC': SVC(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
}


In [18]:
def cacl_metrics(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(classification_report(y_test, y_pred))
    print('Accuracy: ',   accuracy_score(y_test, y_pred))


In [21]:
X_train.shape

(11740, 768)

In [22]:
for model_name, model in models.items():
    print(f"Model: {model_name}")
    cacl_metrics(model, X_train, X_test, y_train, y_test)

    print("-"*100)


Model: Logistic Regression


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.81      0.78      0.79      1543
           1       0.77      0.79      0.78      1393

    accuracy                           0.79      2936
   macro avg       0.79      0.79      0.79      2936
weighted avg       0.79      0.79      0.79      2936

Accuracy:  0.7867847411444142
----------------------------------------------------------------------------------------------------
Model: SVC
              precision    recall  f1-score   support

           0       0.83      0.78      0.80      1543
           1       0.77      0.82      0.79      1393

    accuracy                           0.80      2936
   macro avg       0.80      0.80      0.80      2936
weighted avg       0.80      0.80      0.80      2936

Accuracy:  0.7970027247956403
----------------------------------------------------------------------------------------------------
Model: Random Forest
              precision    recall  f1-score   suppor